In [1]:
# !pip install apimoex

In [2]:
import requests
import apimoex
import pandas as pd
import plotly
import plotly.graph_objs as go
import plotly.express as px


with requests.Session() as session:
    data = apimoex.get_board_history(session, 'GAZP')
    df = pd.DataFrame(data)
    df.set_index('TRADEDATE', inplace = True)
    print(df.head(), '\n')
    print(df.tail(),'\n')
    df.info

           BOARDID   CLOSE    VOLUME         VALUE
TRADEDATE                                         
2014-06-09    TQBR  144.40  40347300  5.851239e+09
2014-06-10    TQBR  144.75  41460990  5.956067e+09
2014-06-11    TQBR  146.40  39418520  5.725373e+09
2014-06-16    TQBR  145.20  77165360  1.113453e+10
2014-06-17    TQBR  144.77  44370530  6.436430e+09 

           BOARDID   CLOSE     VOLUME         VALUE
TRADEDATE                                          
2024-09-19    TQBR  122.24   43382140  5.300176e+09
2024-09-20    TQBR  122.40   28518020  3.496052e+09
2024-09-23    TQBR  130.35  138227980  1.763053e+10
2024-09-24    TQBR  139.71  296528780  4.010794e+10
2024-09-25    TQBR  135.44  150051920  2.059054e+10 



In [3]:
df_n = df[['CLOSE']]
df_n

,CLOSE
TRADEDATE,
2014-06-09,144.40
2014-06-10,144.75
2014-06-11,146.40
2014-06-16,145.20
2014-06-17,144.77
...,...
2024-09-19,122.24
2024-09-20,122.40
2024-09-23,130.35


In [4]:
fig = px.line(df_n,
              #  title = 'Изменение',
                width=1200, height=600,
                      labels={
                     "value": "Стоимость 1 акции, руб.",
                     "TRADEDATE": "Дата проведения торгов",
                     "species": "Species of Iris"
                 }

              )
fig.update_layout(showlegend=False)
fig.show()

Из графика можно видеть отсутствующие значения, когда торги не проводились (в феврале 2022 г.)

In [5]:
def buildLaggedFeatures(df_n,lag=5,dropna=True):
 if type(df_n) is pd.DataFrame:
    new_dict={}
    for col_name in df_n:
        new_dict[col_name]=df_n[col_name]
        for l in range(1,lag+1):
            new_dict['%s_lag%d' %(col_name,l)]=df_n[col_name].shift(l)
    res=pd.DataFrame(new_dict,index=df_n.index)
 else:
    return None
 if dropna:
    return res.dropna()
 else:
    return res

In [6]:
res=buildLaggedFeatures(df_n,lag=10,dropna=True)
res.head()

,CLOSE,CLOSE_lag1,CLOSE_lag2,CLOSE_lag3,CLOSE_lag4,CLOSE_lag5,CLOSE_lag6,CLOSE_lag7,CLOSE_lag8,CLOSE_lag9,CLOSE_lag10
TRADEDATE,,,,,,,,,,,
2014-06-25,149.26,153.25,147.17,145.60,145.20,145.00,144.77,145.20,146.40,144.75,144.40
2014-06-26,147.44,149.26,153.25,147.17,145.60,145.20,145.00,144.77,145.20,146.40,144.75
2014-06-27,149.39,147.44,149.26,153.25,147.17,145.60,145.20,145.00,144.77,145.20,146.40
2014-06-30,148.96,149.39,147.44,149.26,153.25,147.17,145.60,145.20,145.00,144.77,145.20
2014-07-01,148.97,148.96,149.39,147.44,149.26,153.25,147.17,145.60,145.20,145.00,144.77


In [7]:
train=res.loc[:'2021-06-09']
test=res.loc['2021-06-10':]

In [8]:
# !pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

In [9]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "22.0.1" 2024-04-16; OpenJDK Runtime Environment Homebrew (build 22.0.1); OpenJDK 64-Bit Server VM Homebrew (build 22.0.1, mixed mode, sharing)
  Starting server from /Users/stureiko/miniforge3/envs/otus/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/lk/6f78t4jn60s5_ntqqc2dj0980000gn/T/tmpenl8w4uo
  JVM stdout: /var/folders/lk/6f78t4jn60s5_ntqqc2dj0980000gn/T/tmpenl8w4uo/h2o_stureiko_started_from_python.out
  JVM stderr: /var/folders/lk/6f78t4jn60s5_ntqqc2dj0980000gn/T/tmpenl8w4uo/h2o_stureiko_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Istanbul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,"28 days, 3 hours and 32 minutes"
H2O_cluster_name:,H2O_from_python_stureiko_hcr42v
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.984 Gb
H2O_cluster_total_cores:,10
H2O_cluster_allowed_cores:,10
H2O_cluster_status:,"locked, healthy"


In [10]:
hdff = h2o.H2OFrame(train)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [11]:
hdfp = h2o.H2OFrame(test)
hdfp.tail()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


CLOSE,CLOSE_lag1,CLOSE_lag2,CLOSE_lag3,CLOSE_lag4,CLOSE_lag5,CLOSE_lag6,CLOSE_lag7,CLOSE_lag8,CLOSE_lag9,CLOSE_lag10
118.78,121.62,123.44,125.04,123,123.76,122.46,120.6,123.86,122.97,130.27
119.91,118.78,121.62,123.44,125.04,123,123.76,122.46,120.6,123.86,122.97
121.73,119.91,118.78,121.62,123.44,125.04,123,123.76,122.46,120.6,123.86
123.1,121.73,119.91,118.78,121.62,123.44,125.04,123,123.76,122.46,120.6
122.18,123.1,121.73,119.91,118.78,121.62,123.44,125.04,123,123.76,122.46
122.24,122.18,123.1,121.73,119.91,118.78,121.62,123.44,125.04,123,123.76
122.4,122.24,122.18,123.1,121.73,119.91,118.78,121.62,123.44,125.04,123
130.35,122.4,122.24,122.18,123.1,121.73,119.91,118.78,121.62,123.44,125.04
139.71,130.35,122.4,122.24,122.18,123.1,121.73,119.91,118.78,121.62,123.44
135.44,139.71,130.35,122.4,122.24,122.18,123.1,121.73,119.91,118.78,121.62


In [12]:
y = "CLOSE"
x_train = hdff.columns
x_train.remove(y)
aml = H2OAutoML(max_runtime_secs=30, nfolds = 5)
aml.train(x = x_train, y = y, training_frame = hdff)

AutoML progress: |
15:24:15.8: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),6/51
# GBM base models (used / total),5/41
# GLM base models (used / total),1/1
# DRF base models (used / total),0/2
# DeepLearning base models (used / total),0/7
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [13]:
pred = aml.predict(hdfp)
pred.tail()

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict
122.353
119.819
119.87
121.102
122.533
122.359
122.413
122.519
128.208
136.726


In [14]:
aml.leaderboard

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_AllModels_3_AutoML_1_20240926_152414,2.94998,8.7024,2.01487,0.0168567,8.7024
StackedEnsemble_AllModels_4_AutoML_1_20240926_152414,2.95073,8.70679,2.01333,0.0168603,8.70679
StackedEnsemble_BestOfFamily_3_AutoML_1_20240926_152414,2.95168,8.7124,2.01431,0.0168639,8.7124
StackedEnsemble_AllModels_1_AutoML_1_20240926_152414,2.95183,8.71327,2.01523,0.0168677,8.71327
StackedEnsemble_AllModels_2_AutoML_1_20240926_152414,2.9523,8.71609,2.01579,0.0168758,8.71609
StackedEnsemble_BestOfFamily_2_AutoML_1_20240926_152414,2.95325,8.72166,2.01564,0.0168788,8.72166
GLM_1_AutoML_1_20240926_152414,2.96027,8.76321,2.02007,0.0169113,8.76321
StackedEnsemble_BestOfFamily_1_AutoML_1_20240926_152414,2.96272,8.77773,2.02082,0.0169224,8.77773
GBM_lr_annealing_selection_AutoML_1_20240926_152414_select_model,2.9993,8.9958,2.09686,0.0171139,8.9958
GBM_3_AutoML_1_20240926_152414,3.00064,9.00386,2.11165,0.0170904,9.00386


In [17]:
# aml.explain(hdfp)

In [18]:
predict = pred.as_data_frame() # перегоняем обратно в датафрейм

/Users/stureiko/miniforge3/envs/otus/lib/python3.10/site-packages/h2o/frame.py:1981: H2ODependencyWarning:

Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)




In [19]:
r1 = pd.DataFrame(columns=['Дата', 'Значение', 'Тип'])
r1['Дата']=train.index.values
r1['Значение']=train[['CLOSE']].values
r1['Тип'] = 'Обучающая выборка'

In [20]:
r2 = pd.DataFrame(columns=['Дата', 'Значение', 'Тип'])
r2['Дата']=test.index.values
r2['Значение']=test[['CLOSE']].values
r2['Тип'] = 'Тестовая выборка'

In [21]:
r3 = pd.DataFrame(columns=['Дата', 'Значение', 'Тип'])
r3['Дата']=test.index.values
r3['Значение']=predict.values
r3['Тип'] = 'Предсказание модели'

In [22]:
r= pd.concat([r1,r2,r3])


In [23]:
fig = px.line(r, x="Дата", y="Значение", color='Тип',
                width=1200, height=600,
                         labels={
                     "Значение": "Стоимость 1 акции, руб.",
                     "Дата": "Дата проведения торгов",

                 }
              )
fig.update_layout(showlegend=True)
fig.show()

/Users/stureiko/miniforge3/envs/otus/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [24]:
col = res.columns [1:]
day = res.iloc[:,:-1].tail(1).values
day = pd.DataFrame(day, columns=col)
day

,CLOSE_lag1,CLOSE_lag2,CLOSE_lag3,CLOSE_lag4,CLOSE_lag5,CLOSE_lag6,CLOSE_lag7,CLOSE_lag8,CLOSE_lag9,CLOSE_lag10
0,135.44,139.71,130.35,122.4,122.24,122.18,123.1,121.73,119.91,118.78


In [25]:
day = h2o.H2OFrame(day)
day


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


CLOSE_lag1,CLOSE_lag2,CLOSE_lag3,CLOSE_lag4,CLOSE_lag5,CLOSE_lag6,CLOSE_lag7,CLOSE_lag8,CLOSE_lag9,CLOSE_lag10
135.44,139.71,130.35,122.4,122.24,122.18,123.1,121.73,119.91,118.78


In [26]:
# прозноз на 1 день (на завтра)
pred1 = aml.predict(day)
pred1.tail()

stackedensemble prediction progress: |

███████████████████████████████████████████| (done) 100%


predict
136.022
